<a href="https://colab.research.google.com/github/hamin32/Extracurricular-activities/blob/main/%EB%B0%B0%EC%86%A1%EA%B2%BD%EB%A1%9C_%EC%B5%9C%EC%A0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import math
import folium
import numpy as np

# 1️⃣ 데이터 불러오기
path = "/content/gangnam_ctc_dummy_far_40.csv"
df = pd.read_csv(path)

# 2️⃣ 중심점 계산 (픽업 평균값)
origin_lat = df["pickup_lat"].mean()
origin_lon = df["pickup_lon"].mean()

# 3️⃣ 방향 클러스터링 함수
def assign_direction_clusters(df, origin_lat, origin_lon, angle_threshold_deg=25):
    angle_threshold = math.radians(angle_threshold_deg)
    tmp = []

    for idx, row in df.iterrows():
        dx = row["dropoff_lon"] - origin_lon
        dy = row["dropoff_lat"] - origin_lat
        theta = math.atan2(dy, dx)
        r = math.hypot(dx, dy)
        tmp.append((idx, theta, r))

    if not tmp:
        return pd.Series([], dtype=int)

    tmp.sort(key=lambda x: x[1])
    cluster_ids = {}
    current_cluster = 0
    prev_theta = tmp[0][1]
    cluster_ids[tmp[0][0]] = current_cluster

    for idx, theta, r in tmp[1:]:
        if abs(theta - prev_theta) > angle_threshold:
            current_cluster += 1
        cluster_ids[idx] = current_cluster
        prev_theta = theta

    return df.index.map(cluster_ids.get)

# 4️⃣ 클러스터 ID 생성 및 정수형 변환
df["cluster_id"] = assign_direction_clusters(df, origin_lat, origin_lon, angle_threshold_deg=25)
df["cluster_id"] = df["cluster_id"].fillna(0).astype(np.int64)

# 5️⃣ 지도 시각화
m = folium.Map(location=[origin_lat, origin_lon], zoom_start=15)

colors = [
    "red", "blue", "green", "purple", "orange",
    "darkred", "lightblue", "lightgreen", "cadetblue",
    "darkpurple", "pink", "black"
]

for _, row in df.iterrows():
    c = colors[int(row["cluster_id"]) % len(colors)]

    folium.PolyLine(
        locations=[
            (row["pickup_lat"], row["pickup_lon"]),
            (row["dropoff_lat"], row["dropoff_lon"])
        ],
        color=c,
        weight=3,
        opacity=0.7
    ).add_to(m)

    folium.CircleMarker(
        location=(row["pickup_lat"], row["pickup_lon"]),
        radius=3,
        color=c,
        fill=True,
        fill_opacity=0.9
    ).add_to(m)

    folium.CircleMarker(
        location=(row["dropoff_lat"], row["dropoff_lon"]),
        radius=3,
        color=c,
        fill=True,
        fill_opacity=0.6
    ).add_to(m)

# 6️⃣ 바로 지도 출력 (저장 X)
m



In [25]:
import math
import itertools
import pandas as pd
import folium
from IPython.display import display


# =========================
# 설정
# =========================

SPEED_KMPH = 25.0       # 라이더 평균 속도
FUEL_PER_KM = 42.5      # km당 유류비 (원) - 근사치
CSV_PATH = "gangnam_ctc_dummy_40.csv"


# =========================
# 유틸 함수
# =========================

def haversine_km(lat1, lon1, lat2, lon2):
    """위경도 좌표 간 거리(km)."""
    R = 6371.0
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = (math.sin(dphi/2) ** 2
         + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2) ** 2)
    return 2 * R * math.asin(math.sqrt(a))


def rider_gross_payout(distance_km):
    """
    스샷 기준 평시요금에서 라이더 몫(약 85%)만 사용.
    (실제 요금 체계 있으면 여기만 고치면 됨)
    """
    if distance_km <= 2:
        return 2465  # 2900 * 0.85
    elif distance_km <= 3:
        return 2890  # 3400 * 0.85
    elif distance_km <= 4:
        return 3315  # 3900 * 0.85
    elif distance_km <= 5:
        return 3740  # 4400 * 0.85
    else:
        extra_km = math.ceil(distance_km - 5)
        return 3740 + extra_km * int(500 * 0.85)


def single_order_metrics(row):
    """단건 주문 수익/시간 지표."""
    d = haversine_km(row.pickup_lat, row.pickup_lon,
                     row.dropoff_lat, row.dropoff_lon)
    gross = rider_gross_payout(d)
    fuel = FUEL_PER_KM * d
    net = gross - fuel
    time_h = d / SPEED_KMPH
    return {
        "distance_km": d,
        "gross": gross,
        "net": net,
        "time_h": time_h,
        "earn_per_km": net / d if d > 0 else 0,
        "earn_per_h": net / time_h if time_h > 0 else 0,
    }


def best_route_for_bundle(df, job_ids):
    """
    묶음(<=3건)에 대해:
      - pickup/drop 순서 제약 X (A 드롭이 A 픽업보다 먼저 와도 허용)
      - 단, 경로:
          1) 시작은 반드시 픽업(P)
          2) 끝은 반드시 드롭(D)
    조건을 만족하는 순열 중 최단 경로를 찾는다.
    """
    job_ids = [int(j) for j in job_ids]

    points = []
    for jid in job_ids:
        r = df[df.job_id == jid].iloc[0]
        points.append(("p", jid, r.pickup_lat, r.pickup_lon))
        points.append(("d", jid, r.dropoff_lat, r.dropoff_lon))

    best_seq = None
    best_dist = None

    for perm in itertools.permutations(points):
        # 시작은 픽업
        if perm[0][0] != "p":
            continue
        # 끝은 드롭
        if perm[-1][0] != "d":
            continue

        # (픽업 후 드롭 제약은 의도적으로 제거)

        dist = 0.0
        for i in range(len(perm) - 1):
            _, _, lat1, lon1 = perm[i]
            _, _, lat2, lon2 = perm[i + 1]
            dist += haversine_km(lat1, lon1, lat2, lon2)

        if best_dist is None or dist < best_dist:
            best_seq = perm
            best_dist = dist

    return best_seq, best_dist


def bundle_metrics(df, job_ids, single_metrics):
    """묶음(1~3건) 수익/시간 지표."""
    job_ids = [int(j) for j in job_ids]
    gross = sum(single_metrics[j]["gross"] for j in job_ids)

    # 1건짜리는 직선 경로로 빠르게 처리
    if len(job_ids) == 1:
        jid = job_ids[0]
        r = df[df.job_id == jid].iloc[0]
        d = single_metrics[jid]["distance_km"]
        route_seq = (
            ("p", jid, r.pickup_lat, r.pickup_lon),
            ("d", jid, r.dropoff_lat, r.dropoff_lon),
        )
        route_km = d
    else:
        route_seq, route_km = best_route_for_bundle(df, job_ids)

    fuel = FUEL_PER_KM * route_km
    net = gross - fuel
    time_h = route_km / SPEED_KMPH

    return {
        "bundle": tuple(job_ids),
        "size": len(job_ids),
        "route_km": route_km,
        "gross": gross,
        "net": net,
        "time_h": time_h,
        "earn_per_km": net / route_km if route_km > 0 else 0,
        "earn_per_h": net / time_h if time_h > 0 else 0,
        "route_seq": route_seq,
    }


# =========================
# 데이터 로드 & 단건 메트릭
# =========================

df = pd.read_csv(CSV_PATH)
df["job_id"] = df["job_id"].astype(int)

single_metrics = {
    int(r.job_id): single_order_metrics(r)
    for _, r in df.iterrows()
}
jobs = list(df.job_id)


# =========================
# 1·2·3 묶음 후보 생성
# =========================

candidates = []

# 1건 묶음 (직선 경로 사용)
for jid in jobs:
    jid = int(jid)
    m = single_metrics[jid]
    r = df[df.job_id == jid].iloc[0]
    route_seq = (
        ("p", jid, r.pickup_lat, r.pickup_lon),
        ("d", jid, r.dropoff_lat, r.dropoff_lon),
    )
    candidates.append({
        "bundle": (jid,),
        "size": 1,
        "route_km": m["distance_km"],
        "gross": m["gross"],
        "net": m["net"],
        "time_h": m["time_h"],
        "earn_per_km": m["earn_per_km"],
        "earn_per_h": m["earn_per_h"],
        "route_seq": route_seq,
    })

# 2·3건 묶음
for k in (2, 3):
    for combo in itertools.combinations(jobs, k):
        combo = tuple(int(x) for x in combo)
        m = bundle_metrics(df, combo, single_metrics)
        # 경로가 없을 수 있는 edge case 방어
        if m["route_seq"] is not None and m["route_km"] is not None:
            candidates.append(m)

cand_df = pd.DataFrame(candidates)


# =========================
# 탐욕적 선택: 시급(earn_per_h) 최대화하는 disjoint 묶음
# =========================

selected_rows = []
used = set()

for _, row in cand_df.sort_values("earn_per_h", ascending=False).iterrows():
    bundle = row["bundle"]
    if any(j in used for j in bundle):
        continue
    selected_rows.append(row)
    for j in bundle:
        used.add(j)

selected_df = pd.DataFrame(selected_rows).reset_index(drop=True)


# =========================
# 각 묶음에 대해 1/2/3 구조 중 뭐가 유리한지 계산
# =========================

def best_group_size_for_jobs(job_ids):
    job_ids = [int(j) for j in job_ids]

    def eval_part(part):
        if len(part) == 1:
            m = single_metrics[part[0]]
            return m["net"], m["time_h"]
        else:
            m = bundle_metrics(df, part, single_metrics)
            return m["net"], m["time_h"]

    def gen_partitions(remaining):
        if not remaining:
            return [[]]
        res = []
        first = remaining[0]
        rest = remaining[1:]

        # 1개
        for p in gen_partitions(rest):
            res.append([(first,)] + p)

        # 2개
        for i in range(1, len(remaining)):
            pair = tuple(sorted((first, remaining[i])))
            rest2 = [r for idx, r in enumerate(remaining) if idx not in (0, i)]
            for p in gen_partitions(rest2):
                res.append([pair] + p)

        # 3개
        if len(remaining) >= 3:
            for i, j in itertools.combinations(range(1, len(remaining)), 2):
                triple = tuple(sorted((first, remaining[i], remaining[j])))
                rest3 = [r for idx, r in enumerate(remaining) if idx not in (0, i, j)]
                for p in gen_partitions(rest3):
                    res.append([triple] + p)

        # 중복 제거
        norm = []
        seen = set()
        for part in res:
            key = tuple(sorted(tuple(sorted(b)) for b in part))
            if key not in seen:
                seen.add(key)
                norm.append(list(key))
        return norm

    best_eff = -1
    best_part = None

    for part in gen_partitions(job_ids):
        tot_net, tot_time = 0.0, 0.0
        for b in part:
            n, t = eval_part(b)
            tot_net += n
            tot_time += t
        if tot_time == 0:
            continue
        eff = tot_net / tot_time
        if eff > best_eff:
            best_eff = eff
            best_part = part

    size_counts = {1: 0, 2: 0, 3: 0}
    for b in best_part:
        size_counts[len(b)] += 1

    # 가장 많이 쓰인 묶음 크기를 대표 값으로 사용
    best_size = max(size_counts, key=lambda s: (size_counts[s], s))
    return best_size


summary_rows = []
for idx, row in selected_df.iterrows():
    best_size = best_group_size_for_jobs(row["bundle"])
    summary_rows.append({
        "bundle_index": idx,
        "jobs": row["bundle"],
        "size_selected": row["size"],
        "route_km": round(row["route_km"], 3),
        "net_krw": round(row["net"]),
        "earn_per_km": round(row["earn_per_km"], 1),
        "earn_per_h": round(row["earn_per_h"], 1),
        "best_group_size_for_these_jobs": best_size,
    })

summary_df = pd.DataFrame(summary_rows)

print("=== 최종 배차 묶음별 수익 표 ===")
print(summary_df.to_string(index=False))


# =========================
# Folium 지도 출력
# =========================

center_lat = pd.concat([df["pickup_lat"], df["dropoff_lat"]]).mean()
center_lon = pd.concat([df["pickup_lon"], df["dropoff_lon"]]).mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

colors = [
    "red", "blue", "green", "purple", "orange", "darkred", "lightred",
    "beige", "darkblue", "darkgreen", "cadetblue", "darkpurple",
    "white", "pink", "lightblue", "lightgreen", "gray", "black"
]

for idx, row in selected_df.iterrows():
    bundle = row["bundle"]
    color = colors[idx % len(colors)]

    # 마커: 픽업/드롭
    for jid in bundle:
        jid = int(jid)
        r = df[df.job_id == jid].iloc[0]

        # 픽업
        folium.CircleMarker(
            location=[r.pickup_lat, r.pickup_lon],
            radius=4,
            color=color,
            fill=True,
            popup=f"Bundle {idx} P-{jid}"
        ).add_to(m)

        # 드롭
        folium.CircleMarker(
            location=[r.dropoff_lat, r.dropoff_lon],
            radius=4,
            color=color,
            fill=True,
            popup=f"Bundle {idx} D-{jid}"
        ).add_to(m)

    # 경로 polyline
    if len(bundle) == 1:
        # 단건: 직선 경로
        jid = int(bundle[0])
        r = df[df.job_id == jid].iloc[0]
        line = [
            (r.pickup_lat, r.pickup_lon),
            (r.dropoff_lat, r.dropoff_lon),
        ]
    else:
        seq, _ = best_route_for_bundle(df, bundle)
        if seq is None:
            continue
        line = [(lat, lon) for (_, _, lat, lon) in seq]

    folium.PolyLine(
        locations=line,
        color=color,
        weight=3,
        opacity=0.8
    ).add_to(m)

# 노트북에 바로 표시 + 파일로도 저장
m.save("bundles_map.html")
display(m)


=== 최종 배차 묶음별 수익 표 ===
 bundle_index         jobs  size_selected  route_km  net_krw  earn_per_km  earn_per_h  best_group_size_for_these_jobs
            0        (34,)              1     0.317     2452       7744.3    193607.0                               1
            1        (39,)              1     0.334     2451       7327.7    183193.7                               1
            2        (14,)              1     0.513     2443       4763.2    119079.1                               1
            3        (35,)              1     0.538     2442       4535.5    113387.9                               1
            4        (19,)              1     0.560     2441       4358.3    108957.4                               1
            5      (3, 13)              2     1.164     4881       4192.5    104811.5                               2
            6     (22, 26)              2     1.188     4880       4107.2    102681.0                               2
            7         (5,)       